In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics

In [2]:
burns=pd.read_csv("data2/surgery.csv")

In [3]:
apachepredvar = pd.read_csv('apachePredVar.csv.gz')

In [4]:
burns_merged = pd.merge(apachepredvar[['patientunitstayid','pao2','fio2']], burns, how='right', on='patientunitstayid')

In [5]:
burns_merged['sofa_ratio'] = 0

In [6]:
for i in range(len(burns_merged)):
    if (burns_merged.iloc[i]['pao2'] != -1) & (burns_merged.iloc[i]['fio2'] != -1):
        if ((burns_merged.iloc[i]['pao2']/burns_merged.iloc[i]['fio2']) * 100 <= 399) & ((burns_merged.iloc[i]['pao2']/burns_merged.iloc[i]['fio2']) * 100 >= 300):
            burns_merged['sofa_ratio'].iloc[i] = 1
        if ((burns_merged.iloc[i]['pao2']/burns_merged.iloc[i]['fio2']) * 100 <= 299) & ((burns_merged.iloc[i]['pao2']/burns_merged.iloc[i]['fio2']) * 100 >= 200):
            burns_merged['sofa_ratio'].iloc[i] = 2
        if ((burns_merged.iloc[i]['pao2']/burns_merged.iloc[i]['fio2']) * 100 <= 199) & (burns_merged.iloc[i]['ventday1'] != 1):
            burns_merged['sofa_ratio'].iloc[i] = 2
        if ((burns_merged.iloc[i]['pao2']/burns_merged.iloc[i]['fio2']) * 100 >= 100) & ((burns_merged.iloc[i]['pao2']/burns_merged.iloc[i]['fio2']) * 100 <= 199) & (burns_merged.iloc[i]['ventday1'] == 1):
            burns_merged['sofa_ratio'].iloc[i] = 3
        if ((burns_merged.iloc[i]['pao2']/burns_merged.iloc[i]['fio2']) * 100 < 100) & (burns_merged.iloc[i]['ventday1'] == 1):
            burns_merged['sofa_ratio'].iloc[i] = 4

C:\Users\nsafa\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [7]:
def platelets(item):
    temp = 0
    if (item >= 100) & (item <= 149):
        temp += 1
    elif (50 <= item) & (item <= 99):
        temp += 2
    elif (20 <= item) & (item <= 49):
        temp += 3
    elif (20 > item):
        temp += 4
    return temp

In [8]:
burns_merged['sofa_platelets'] = burns_merged['platelets x 1000'].apply(platelets)

In [9]:
burns_merged['GCS'] = burns_merged['eyes'] + burns_merged['motor'] + burns_merged['verbal']

In [10]:
def GCS(item):
    temp = 0
    if (item >= 13) & (item <= 14):
        temp += 1
    elif (10 <= item) & (item <= 12):
        temp += 2
    elif (6 <= item) & (item <= 9):
        temp += 3
    elif (6 > item):
        temp += 4
    return temp

In [11]:
burns_merged['sofa_GCS'] = burns_merged['GCS'].apply(GCS)

In [12]:
apacheapsdvar = pd.read_csv('ApacheApsVar.csv.gz')

In [13]:
burns_merged_1 = pd.merge(apacheapsdvar[['patientunitstayid','bilirubin']], burns_merged, how='right', on='patientunitstayid')

In [14]:
def bilirubin(item):
    temp = 0
    if (item >= 1.2) & (item <= 1.9):
        temp += 1
    elif (2 <= item) & (item <= 5.9):
        temp += 2
    elif (6 <= item) & (item <= 11.9):
        temp += 3
    elif (12 <= item):
        temp += 4
    return temp

In [15]:
burns_merged_1['sofa_bilirubin'] = burns_merged_1['bilirubin'].apply(bilirubin)

In [16]:
def meanbp(item):
    temp = 0
    if (item < 70):
        temp += 1
    return temp

In [17]:
burns_merged_1['sofa_meanbp'] = burns_merged_1['meanbp'].apply(meanbp)

In [18]:
burns_merged_1['creatinine'].mean()

1.2628464713370715

In [19]:
apacheapsdvar = pd.read_csv('ApacheApsVar.csv.gz')

In [20]:
burns_merged_2 = pd.merge(apacheapsdvar[['patientunitstayid','urine']], burns_merged_1, how='right', on='patientunitstayid')

In [21]:
burns_merged_2['sofa_creatinine_urine'] = 0

In [22]:
for i in range(len(burns_merged_2)):
    if (burns_merged_2['creatinine'].iloc[i] >= 1.2) & (burns_merged_2['creatinine'].iloc[i] <= 1.9):
        burns_merged_2['sofa_creatinine_urine'].iloc[i] = 1
    elif (burns_merged_2['creatinine'].iloc[i] >= 2) & (burns_merged_2['creatinine'].iloc[i] <= 3.4):
        burns_merged_2['sofa_creatinine_urine'].iloc[i] = 2
    elif ((burns_merged_2['creatinine'].iloc[i] >= 3.5) & (burns_merged_2['creatinine'].iloc[i] <= 4.9)) | ((burns_merged_2['urine'].iloc[i] < 500) & (burns_merged_2['urine'].iloc[i] != -1)):
        burns_merged_2['sofa_creatinine_urine'].iloc[i] = 3
    elif (burns_merged_2['creatinine'].iloc[i] >= 5) | ((burns_merged_2['urine'].iloc[i] < 200) & (burns_merged_2['urine'].iloc[i] != -1)):
        burns_merged_2['sofa_creatinine_urine'].iloc[i] = 4

In [23]:
burns_merged_2['sofa'] = burns_merged_2[['sofa_ratio', 'sofa_platelets', 'sofa_GCS', 'sofa_bilirubin',
       'sofa_meanbp', 'sofa_creatinine_urine']].sum(axis=1)

In [24]:
def sofa_mortality(item):
    if (0 <= item) & (item <=1):
        return 0
    elif (2 <= item) & (item <=3):
        return 1.5
    elif (4 <= item) & (item <=5):
        return 6.7
    elif (6 <= item) & (item <=7):
        return 18.2
    elif (8 <= item) & (item <=9):
        return 26.3
    elif (10 <= item) & (item <=11):
        return 45.8
    elif (12 <= item) & (item <=14):
        return 80
    elif (14 < item):
        return 89.7

In [25]:
burns_merged_2['sofa_mortality'] = burns_merged_2['sofa'].apply(sofa_mortality)

In [26]:
burns_merged_2

,patientunitstayid,urine,bilirubin,pao2,fio2,gender,age,ethnicity,apacheadmissiondx,admissionheight,...,anion gap,sofa_ratio,sofa_platelets,GCS,sofa_GCS,sofa_bilirubin,sofa_meanbp,sofa_creatinine_urine,sofa,sofa_mortality
0,142476,-1.0000,-1.0,-1.0,-1.0,Male,78,Caucasian,"Knee replacement, total (non-traumatic)",175.3,...,9.733549,0,0,15.0,0,0,0,1,1,0.0
1,143274,-1.0000,-1.0,-1.0,-1.0,Male,22,Caucasian,Head/extremity trauma,195.6,...,11.500000,0,0,13.0,1,0,0,1,2,1.5
2,143573,-1.0000,0.3,-1.0,-1.0,Male,80,Caucasian,"Hip replacement, total (non-traumatic)",175.2,...,15.000000,0,0,15.0,0,0,0,2,2,1.5
3,144173,-1.0000,0.7,46.0,50.0,Female,40,Asian,"Genitourinary surgery, other",154.9,...,20.750000,4,1,15.0,0,0,1,1,7,18.2
4,144275,-1.0000,-1.0,-1.0,-1.0,Male,66,Caucasian,"Aneurysm/pseudoaneurysm, other",167.6,...,9.733549,0,0,15.0,0,0,0,1,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2650,3351763,3344.5440,0.4,104.0,60.0,Female,62,Caucasian,"Fistula/abscess, surgery for (not inflammatory...",165.1,...,9.000000,3,0,10.0,2,0,1,0,6,18.2
2651,3352092,2220.9984,1.6,-1.0,-1.0,Male,64,Caucasian,"Vascular surgery, other",160.0,...,3.000000,0,2,15.0,0,1,0,0,3,1.5
2652,3352757,823.1328,-1.0,-1.0,-1.0,Female,64,Caucasian,Complications of prev. peripheral vasc. surger...,162.6,...,6.000000,0,0,15.0,0,0,0,0,0,0.0
2653,3353043,1977.9552,-1.0,78.0,28.0,Female,27,African American,Cesarean section,165.1,...,9.733549,2,0,15.0,0,0,0,1,3,1.5


In [27]:
burns_merged_2['actualicumortality_label'] = burns_merged_2['actualicumortality'].apply(lambda x: 1 if x =='EXPIRED' else 0)

In [28]:
fpr, tpr, thresholds = metrics.roc_curve(burns_merged_2['actualicumortality_label'].values, burns_merged_2['sofa_mortality'].values, pos_label=1)

In [29]:
metrics.auc(fpr, tpr)

0.7718807932138644